# Project 2 - Machine Learning, Fall 2022

Deadline: 11th of December 2022, 23:59

To do this project you have to complete this Jupyter notebook and send it via Discord.

The total number of points allocated for this project is 10.

You will need the following modules to solve the tasks:

In [2]:
import math
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
from sklearn.naive_bayes import BernoulliNB

In [3]:
penguin_dataset = pd.read_csv("data/penguins_filtered.csv")

penguin_dataset = penguin_dataset.replace({
    "Adelie": 1,
    "Chinstrap" : 2,
    "Gentoo": 3,
    "male" : 0,
    "female" : 1,
    "Biscoe" : 0,
    "Dream" : 1,
    "Torgersen" : 2})

discrete_penguin_dataset = penguin_dataset[["sex", "island", "species"]]
print(penguin_dataset.head())
print(discrete_penguin_dataset.head())

   sex  island  bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g  \
0    0       2            39.1           18.7              181.0       3750.0   
1    1       2            39.5           17.4              186.0       3800.0   
2    1       2            40.3           18.0              195.0       3250.0   
3    1       2            36.7           19.3              193.0       3450.0   
4    0       2            39.3           20.6              190.0       3650.0   

   species  
0        1  
1        1  
2        1  
3        1  
4        1  
   sex  island  species
0    0       2        1
1    1       2        1
2    1       2        1
3    1       2        1
4    0       2        1


## I. Naive and Joint Bayes (3.5 points; 0.15 bonus per week)

1. Calculate the prior probabilities for the target feature. Transform them by applying the natural logarithm. 

In [4]:
# solution here
def prior_probability_target(dataset, target):
    prob = [ list(dataset[target]).count(val) / len(dataset[target]) for val in np.unique(dataset[target])]
    prob = list(map(lambda x: math.log(x), prob))
    return prob

prior_probability_target(penguin_dataset, 'species')

[-0.8245358682721073, -1.5886347848043372, -1.0290189968689145]

2. Write the formulas used to calculate the maximum aposteriori probability using Naive Bayes and Joint Bayes. Use the names of the variables from the discrete penguin dataset.

*Answer here*:

**Naive Bayes:** 
* s = argmax P(sex|species = s) * P(island|species = s) * P(species = s)

**Joint Bayes:** 
* s = argmax P(sex, island | species = s) * P(species = s)

3. Find and calculate the logarithm of all the conditional probabilities (also names likelihoods) used to predict the label for the instance `{"sex" : 1, "island" : 2}` in Naive Bayes.

In [5]:
# solution here
def likelihoods(dataset, target, sex_value, island_value):
    probabilities = list()

    for species_value in np.unique(dataset[target]):
        ds = dataset[dataset[target] == species_value].reset_index(drop=True)

        sex_count = list(ds['sex']).count(sex_value)
        island_count = list(ds['island']).count(island_value)
        species_count = list(dataset[target]).count(species_value)

        probabilities.append([sex_count/species_count, island_count/species_count])
    return probabilities

likelihoods(discrete_penguin_dataset, 'species', 1, 2)

[[0.5, 0.3219178082191781], [0.5, 0.0], [0.48739495798319327, 0.0]]

4. Why does the results contain infinity? Fix the calculation by using the Laplace Smoothing with `alpha = 1`.

In [6]:
# solution here
def likelihoods_laplace(dataset, target, sex_value, island_value, alpha = 1):
    probabilities = list()

    for species_value in np.unique(dataset[target]):
        ds = dataset[dataset[target] == species_value].reset_index(drop=True)

        sex_count = list(ds['sex']).count(sex_value) + alpha
        island_count = list(ds['island']).count(island_value) + alpha
        species_count = len(ds['species'])

        probabilities.append ([math.log(sex_count    / (species_count + alpha*len(np.unique(dataset['sex'])))),
                               math.log(island_count / (species_count + alpha*len(np.unique(dataset['island']))))] )
    return probabilities

likelihoods_laplace(discrete_penguin_dataset, 'species', 1, 2)

[[-0.6931471805599453, -1.1327452950375683],
 [-0.6931471805599453, -4.2626798770413155],
 [-0.7182531016910216, -4.804021044733257]]

5. Calculate the aposteriori probabilities of the labels and decide which label will Naive Bayes predict for the instance. Use only the logarithm values.

In [7]:
# solution here
def aposteriori(dataset, target, sex_value, island_value):
    priori = prior_probability_target(dataset, target)
    likelihood = likelihoods_laplace(dataset, target, sex_value, island_value)
    
    prod = list()
    for index in range(len(priori)):
        prod.append( (index + 1, np.prod(likelihood[index]) * priori[index]) )
    return prod
    
aposteriori(discrete_penguin_dataset, 'species', 1 , 2)

[(1, -0.6473919289272929), (2, -4.693882863131364), (3, -3.550633152185177)]

6. **Naive Bayes implemenation**: write a function called `naive_bayes` that takes three arguments:
- `df`: the dataframe which will be used for training
- `index_target`: the index of the column associated with the target feature
- `alpha`: the parameter used for Laplace Smoothing

The function should return a dictionary with the following fields:
- `log_prior`: the logarithmic values of the prior probabilities (the probability of the labels)
- `log_likelihoods`: a n x m x t array, where n - the number of features; m - the number of labels; t - the number of values for a feature; this array will contain the logarithmic values of the likelihoods (P(feature = value | target_feature = label))
- `n_classes`: the number of labels
- `n_feature_classes`: a vector that contains the number of unique values for each attribute
- `classes`: the name of the labels (the values of the target feature).

In [8]:
def naive_bayes(df, index_target = -1, alpha = 1e-10):
    target = df.columns[index_target]
    output = dict()
    output['log_prior'] = prior_probability_target(df, target)
    output['log_likelihoods'] = 'pass'
    output['n_classes'] = np.unique(df[target])
    output['n_feature_classes'] = [ list(np.unique(df[col])) for col in df.columns[:-1] ]
    output['classes'] = list(np.unique(df[target]))

    return output

naive_bayes(discrete_penguin_dataset)

{'log_prior': [-0.8245358682721073, -1.5886347848043372, -1.0290189968689145],
 'log_likelihoods': 'pass',
 'n_classes': array([1, 2, 3], dtype=int64),
 'n_feature_classes': [[0, 1], [0, 1, 2]],
 'classes': [1, 2, 3]}

7. Train the discrete penguin dataset using your version of Naive Bayes and sklearn's. Compare the values of the parameters.(Be careful at what type of Naive Bayes classifier you pick from sklearn!)

In [9]:
# solution here
X = discrete_penguin_dataset[['sex','island']]
y = discrete_penguin_dataset['species']
cl = BernoulliNB(alpha=1e-10).fit(X, y)

new_messages = pd.DataFrame(
  [(1, 2)],
columns = ['sex','island'])

# print(cl.class_log_prior_)
# print(cl.feature_log_prob_)
# print(cl.predict(new_messages), '\n', cl.predict_proba(new_messages))

print(cl.predict_proba(new_messages))

[[6.00000000e-01 4.00000000e-01 5.73405833e-13]]


8. Create a function named `nb_predict_prob` that uses the log probabilities calculated by Naive Bayes to infer the aposteriori probability of a new instance `X`.

In [10]:
def nb_predict_prob(nb_dict, X, use_log = False):
    pass

9. Create a function that does the Naive Bayes prediction using the Maximum Aposteriori Probability.

In [11]:
def nb_predict(nb_dict, X):
    pass

10. Create a function that calculate the accuracy of the trained model on a set of instances `X` with known labels `y`.

In [12]:
def nb_score(nb_dict, X, y):
    pass

11. Calculate the training accuracy of your Naive Bayes algorithm. Explain the results.

In [13]:
# solution here

12. Find and calculate all the conditional probabilities (also names likelihoods) used to predict the label for the instance `{"sex" : 1, "island" : 2}` in Joint Bayes. (*Hint*: panda's query function might provide itself useful.)

In [14]:
# solution here

13. Calculate the aposteriori probabilities of the labels and decide which label will Joint Bayes predict for the instance. Use the conditional and prior probabilities (*not* the logarithmic values).

In [15]:
# solution here

14. **Joint Bayes implemenation**: write a function called `joint_bayes` that takes two arguments:
- `df`: the dataframe which will be used for training
- `index_target`: the index of the column associated with the target feature

The function should return a dictionary with the following fields:
- `prior_probs`: the prior probabilities (the probability of the labels)
- `likelihoods`: a n x m array, where n - the number of labels; m - the number of combination between the values of the features; each label will have assigned a list containing the joint probability P(feature_1 = value_1, feature_2 = value_2, ...,  feature_n = value_n | target_feature = label)
- `n_classes`: the number of labels
- `n_feature_classes`: a vector that contains the number of unique values for each attribute
- `classes`: the name of the labels (the values of the target feature).

*Hint*: check the imports from the first cell of this notebook.

In [16]:
def joint_bayes(df, index_target = -1):
    pass

15. Train the Joint Bayes algorithm on the discrete penguin dataset. Print the obtained dictionary.

In [17]:
# solution here

16. Similarly to Naive Bayes, write the functions used to predict the aposteriori probabilities, the label and the accuracy of the Joint Bayes algorithm.

In [18]:
def jb_predict_prob(jb_dict, X):
    pass

def jb_predict(jb_dict, X):
    pass

def jb_score(jb_dict, X, y):
    pass


17. Calculate the training accuracy of your Joint Bayes algorithm. 

In [19]:
# solution here

## II. kNN (2 points; 0.15 bonus per week)

For this section we will use the entire `penguin_dataset`.

1. Calculate the Euclidean distance between the test instance `{"sex" : 1, "island" : 2, "bill_length" : 20, "bill_depth" : 40, "flipper_length" : 355, "body_mass" : 855}` and the instances from the dataset. Store the values in an object called `instance_distance`. Print the average distance. (*Hint*: check the norm function from the numpy package.)

In [25]:
# solution here
from numpy.linalg import norm

def distances(dataset, test, p):
    instance_distance = list()
    for index in range(len(dataset)):
        line = dataset[list(dataset.columns[:-1])].loc[index]
        instance_distance.append( (dataset.loc[index], norm(test - line, p)) )

    instance_distance.sort(key=lambda x: x[1])
    return instance_distance

6121.9037464596395
   sex  island  bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g
0    0       2            39.1           18.7              181.0       3750.0
1    1       2            39.5           17.4              186.0       3800.0
2    1       2            40.3           18.0              195.0       3250.0
3    1       2            36.7           19.3              193.0       3450.0
4    0       2            39.3           20.6              190.0       3650.0


2. Find the `5` nearest neighbours of the test instance.

In [20]:
# solution here
first_five = distances(penguin_dataset, [1, 2, 20, 40, 355, 855], 2)[:5]
first_five

[(sex                     1.0
  island                  1.0
  bill_length_mm         46.9
  bill_depth_mm          16.6
  flipper_length_mm     192.0
  body_mass_g          2700.0
  species                 2.0
  Name: 303, dtype: float64,
  1852.5296677786296),
 (sex                     1.0
  island                  0.0
  bill_length_mm         36.4
  bill_depth_mm          17.1
  flipper_length_mm     184.0
  body_mass_g          2850.0
  species                 1.0
  Name: 58, dtype: float64,
  2002.5142621214961),
 (sex                     1.0
  island                  0.0
  bill_length_mm         36.5
  bill_depth_mm          16.6
  flipper_length_mm     181.0
  body_mass_g          2850.0
  species                 1.0
  Name: 52, dtype: float64,
  2002.7792714126037),
 (sex                     1.0
  island                  2.0
  bill_length_mm         38.6
  bill_depth_mm          17.0
  flipper_length_mm     188.0
  body_mass_g          2900.0
  species                 1.0
  Name

3. Determine the probabilities of the labels that kNN would assign for this test instance (`k = 5`). Which label has the highest probability?

In [21]:
# solution here
def prob_no_weights(dataset, target, k, test_x, p):
    first_k = distances(penguin_dataset, test_x, p)[:k]
    prob_labels = [0] * len(np.unique(dataset[target]))
    for index in range(len(first_k)):
        label = int(first_k[index][0]['species'])
        prob_labels[label- 1] = prob_labels[label - 1] + 1

    prob_labels = list(map(lambda x: x / k, prob_labels))
    return prob_labels

prob_no_weights(penguin_dataset, 'species', 5, [1, 2, 20, 40, 355, 855], 2)


[0.8, 0.2, 0.0]

4. Suppose that kNN gives for each neighbour a weight that is equal to the inverse of the distance. Print the changed probabilities, as well as the label predicted by kNN.

In [22]:
def prob_with_weights(dataset, target, k, test_x, p):
    first_k = distances(penguin_dataset, test_x, p)[:k]
    prob_labels = [0] * len(np.unique(dataset[target]))
    weights = 0
    for index in range(len(first_k)):
        label = int(first_k[index][0]['species'])
        distance = first_k[index][1]
        prob_labels[label- 1] = prob_labels[label - 1] + distance**(-1)
        weights = weights + distance**(-1)

    prob_labels = list(map(lambda x: x / weights, prob_labels))
    return prob_labels

prob_with_weights(penguin_dataset, 'species', 5, [1, 2, 20, 40, 355, 855], 2)

[0.7852063478657878, 0.2147936521342122, 0.0]

5. **k-NN implementation**: Create a function `knn_predict_prob` that will take six arguments:
- `df`: the dataframe containing the features and the target feature
- `test_x`: a list with the attributes observed for *one* instance
- `k`: the number of nearest neighbours
- `use_weights`: boolean value that indicates whether to assign weights based on the inverse of the distance or not
- `p`: either an integer, indicating the order of the Minkowski distance (p=2 is the equivalent for Euclidean) or a custom distance function
- `index_target`: the index of the column that contains the labels of the target feature

The function should:
- calculate the distance between `test_x` and the observations from the dataset
- extract the k-nearest neighbours
- calculate the weight for each label
- normalize the weights to become probabilities
- return the probability vector, that will indicate the probability of the instance `test_x` to have the label `i`.

In [23]:
def knn_predict_prob(df, test_x, k, use_weights = True, p = 2, index_target = -1):
   target = df.columns[index_target]
   if use_weights:
      return prob_with_weights(df, target, k, test_x, p)
   else:
      return prob_no_weights(df, target, k, test_x, p)

knn_predict_prob(penguin_dataset, [1, 2, 20, 40, 355, 855], 5, True, 2, -1)

[0.7852063478657878, 0.2147936521342122, 0.0]

6. Write a function that, based on the probabilities calculated above, returns the label with the highest probability.

In [24]:
def knn_predict(df, test_x, k, use_weights = True, p = 2, index_target = -1):
    probabilities = knn_predict_prob(df, test_x, k, use_weights, p, index_target)
    max_value = max(probabilities)
    return probabilities.index(max_value) + 1

knn_predict(penguin_dataset, [1, 2, 20, 40, 355, 855], 5, True, 2, -1)

1

7. Calculate the probabilities and the predicted label for the instance from exercise 1 using the following configurations:
- `k = 11, unweighted, Euclidean distance`
- `k = 11, weighted, Euclidean distance`
- `k = 11, unweighted, Manhattan distance`
- `k = 11, weighted, Manhattan distance`

Compare your results with the results obtained by the `sklearn` implementation.

In [25]:
# solution here
print(f'k = 11, unweighted, Euclidean distance: {knn_predict(penguin_dataset, [1, 2, 20, 40, 355, 855], 11, False, 2, -1)}')
print(f'k = 11, weighted, Euclidean distance: {knn_predict(penguin_dataset, [1, 2, 20, 40, 355, 855], 11, True, 2, -1)}')
print(f'k = 11, unweighted, Manhattan distance: {knn_predict(penguin_dataset, [1, 2, 20, 40, 355, 855], 11, False, 1, -1)}')
print(f'k = 11, weighted, Manhattan distance: {knn_predict(penguin_dataset, [1, 2, 20, 40, 355, 855], 11, True, 1, -1)}')

k = 11, unweighted, Euclidean distance: 1
k = 11, weighted, Euclidean distance: 1
k = 11, unweighted, Manhattan distance: 1
k = 11, weighted, Manhattan distance: 1


8. Write a function that calculates the accuracy of the kNN algorithm.

In [30]:
def knn_score(df, test_x, test_y, k, use_weights = True, p = 2, index_target = -1):
    count = 0

    for index in range(len(test_x)):
        attr = list(test_x.loc[index])
        trg = test_y.loc[index]
        predicted = knn_predict(df, attr, k, use_weights, p, index_target)
        if predicted == trg:
            count += 1
    return count/len(test_y)

test_x = penguin_dataset[list(penguin_dataset.columns[:-1])]
test_y = penguin_dataset[penguin_dataset.columns[-1]]
knn_score(penguin_dataset, test_x, test_y, 3, False, 1, -1)

0.924924924924925

9. What is the training accuracy of the unweighted kNN when k varies from 3 to 15? (use only odd numbers). Does adding the weight / changing the distance metric improve the score? Justify.

In [31]:
# solution here
for k in range(3,16,2):
    print(f'k = {k} p = 1: {knn_score(penguin_dataset, test_x, test_y, k, False, 1, -1)}')
    print(f'k = {k} p = 2: {knn_score(penguin_dataset, test_x, test_y, k, False, 2, -1)}')

k = 3 p = 1: 0.924924924924925
k = 3 p = 2: 0.9159159159159159
k = 5 p = 1: 0.8378378378378378
k = 5 p = 2: 0.8378378378378378
k = 7 p = 1: 0.8498498498498499
k = 7 p = 2: 0.8438438438438438
k = 9 p = 1: 0.8228228228228228
k = 9 p = 2: 0.8138138138138138
k = 11 p = 1: 0.8078078078078078
k = 11 p = 2: 0.8018018018018018
k = 13 p = 1: 0.8228228228228228
k = 13 p = 2: 0.8198198198198198
k = 15 p = 1: 0.7897897897897898
k = 15 p = 2: 0.7897897897897898


## III. AdaBoost (3 points; 0.25 bonus per week)

## IV. Testing the performance (1.5 points; 0.1 bonus per week)

# Exercise grading

| Section | Exercise | Points |
| --- | --- | --- |
| I |	1 | 	0.1 |
| I |	2 | 	0.1 |
| I |	3 | 	0.25 |
| I |	4 | 	0.25 |
| I |	5 | 	0.15 |
| I |	6 | 	0.65 |
| I |	7 | 	0.1 |
| I |	8 | 	0.25 |
| I |	9 | 	0.15 |
| I |	10 | 	0.1 |
| I |	11 | 	0.1 |
| I |	12 | 	0.1 |
| I |	13 | 	0.15 |
| I |	14 | 	0.65 |
| I |	15 | 	0.1 |
| I |	16 | 	0.2 |
| I |	17 | 	0.1 |
|II | 	1 | 	0.2 |
|II | 	2 | 	0.1 |
|II | 	3 | 	0.15 |
|II | 	4 | 	0.2 |
|II | 	5 | 	0.72 |
|II | 	6 | 	0.1 |
|II | 	7 | 	0.28 |
|II | 	8 | 	0.1 |
|II | 	9 | 	0.15 |
